In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings("ignore")

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0'}

In [3]:
start_time = time.time()

result_df1 = pd.DataFrame(columns=['Название вакансии', 'з/п', 'город', 'описание вакансии', 'дата размещения', 'ссылка'])
num_of_page = 2

with webdriver.Chrome() as browser:
    for i in range(num_of_page):
        url = f'https://nn.hh.ru/search/vacancy?L_is_autosearch=false&ored_clusters=true&employer_id=3529&page={i+1}'
        
        browser.get(url)
        time.sleep(1)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        links_ = soup.find_all('a', class_='serp-item__title')
        links = [link['href'] for link in links_]

        for link in links:
            result = requests.get(link, headers=headers)
            content = result.text 
            soup = BeautifulSoup(content, 'html.parser')

            name_vac = soup.find('h1', class_='bloko-header-section-1')
            name_vac = name_vac.get_text() if name_vac else 'No information'

            sal = soup.find('div', class_='wrapper-flat--H4DVL_qLjKLCo1sytcNI')
            salary_text = sal.find('span', class_='bloko-header-section-2 bloko-header-section-2_lite').text.strip() if sal and sal.find('span', class_='bloko-header-section-2 bloko-header-section-2_lite') else 'No information'
            salary = salary_text.split('до вычета налогов')[0].strip()

            city_element = soup.find('a', class_='bloko-link bloko-link_kind-tertiary bloko-link_disable-visited')
            city = city_element.get_text() if city_element else soup.find('div', class_='vacancy-company-redesigned').find_all('p')[-1].get_text() if soup.find('div', class_='vacancy-company-redesigned') else 'No information'

            description_el = soup.find('div', class_='vacancy-branded-user-content')
            description = description_el.find_all('p')[1].get_text() if description_el else 'No information'

            date_el = soup.find('p', class_='vacancy-creation-time-redesigned')
            date = date_el.find('span').get_text().replace('₽', '').strip() if date_el else 'No information'

            link_vac = soup.find('link', href=True, rel="canonical")['href']
            
            new_row = {'Название вакансии': name_vac, 'з/п': salary, 'город': city, 'описание вакансии': description, 'дата размещения': date, 'ссылка': link_vac}
            result_df1 = result_df1.append(new_row, ignore_index=True)

end_time = time.time()
execution_time = end_time - start_time
#print(f"Время выполнения: {execution_time} сек.")

#print(result_df1)

# result_df1.to_excel('r.xlsx')

The chromedriver version (114.0.5735.90) detected in PATH at C:\chromedriver\chromedriver.exe might not be compatible with the detected chrome version (115.0.5790.170); currently, chromedriver 115.0.5790.170 is recommended for chrome 115.*, so it is advised to delete the driver in PATH and retry
